## Echem

In [ ]:
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path as path
import sys
import numpy as np
import pandas as pd
import xarray as xr
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import matplotlib.gridspec as gridspec
from matplotlib import ticker
import re
import seaborn as sns

In [ ]:
# 画图的初始设置
plt.style.use(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-python\Figure\liuchzzyy.mplstyle')
# display(plt.style.available)

# 颜色设定
sys.path.append(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Python\Figure')
from colors import tol_cmap, tol_cset
colors = list(tol_cset('vibrant'))
if r'sunset' not in plt.colormaps():
    plt.colormaps.register(tol_cmap('sunset'))
if r'rainbow_PuRd' not in plt.colormaps():
    plt.colormaps.register(tol_cmap('rainbow_PuRd')) # 备用 plasma

# 设置数学字体 Set math font
mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.rm'] = 'Arial'
mpl.rcParams['mathtext.it'] = 'Arial:italic'
mpl.rcParams['mathtext.bf'] = 'Arial:bold'
mpl.rcParams['mathtext.sf'] = 'Arial'
mpl.rcParams['mathtext.tt'] = 'Arial'
mpl.rcParams['mathtext.cal'] = 'Arial'
mpl.rcParams['mathtext.default'] = 'regular'

# 输出的文件夹
path_out = path(r"C:\Users\chengliu\Desktop\Figure")

### αMnO2

#### Special Testings

##### Siavash buffer solution

In [ ]:
path_dir = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\RawData\Echem\αMnO2\SpecialTest\pHBuffer\HAC-NH3-pH4.2+0.5M ZnSO4 + 0.2M MnSO4')
path_filelist = list(path_dir.glob('**/*.txt'))
file_labels = [f.stem for f in path_filelist]  # 获取文件名（无后缀）
file_map = dict(enumerate(file_labels))        # file_idx → 文件名映射

# === 1. 读取所有数据 ===
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', decimal='.', index_col=None).dropna(axis=1, how='all')
    df[['Ewe/V', 'Ece/V', '<I>/mA', 'Capacity/mA.h']] = df[['Ewe/V', 'Ece/V', '<I>/mA', 'Capacity/mA.h']].apply(pd.to_numeric, errors='coerce')
    df['time/s'] = df['time/s'].apply(pd.to_datetime, format='mixed', errors='coerce')
    df['cycle number'] = df['cycle number'].astype(float).astype(np.int16)
    echem.append(df[df['cycle number'].isin([0, 1])])  # 保留第0和1圈

# === 2. 分割放电/充电段 ===
def split_by_current_direction(df: pd.DataFrame, current_col='<I>/mA', cycle_col='cycle number') -> dict:
    result = {}
    grouped = df.groupby(cycle_col)
    for cycle_id, group in grouped:
        group = group.sort_values('time/s')
        current_sign = group[current_col].apply(lambda x: 'charge' if x > 0 else ('discharge' if x < 0 else 'rest'))
        current_state = current_sign.ne(current_sign.shift()).cumsum()
        for seg_id, seg in group.groupby(current_state):
            state = current_sign.iloc[seg.index[0]]
            if state != 'rest':
                key = f'{cycle_id}_{state}'
                result[key] = seg.reset_index(drop=True)
    return result

# === 3. 建立全局数据字典 ===
echem_all = {}
for i, df in enumerate(echem):
    split_dict = split_by_current_direction(df)
    for k, v in split_dict.items():
        cycle, mode = k.split('_')
        new_key = f'file{i}_#{cycle}_{mode}'
        echem_all[new_key] = v

for k in list(echem_all.keys())[:3]:
    print(f'{k}: {echem_all[k].shape}')

In [ ]:
%matplotlib ipympl
# 参数设定
selected_cycles_per_file = {
    0: [0],
    1: [0, 1],
    2: [0, 1],
}

plot_mode = "discharge" # ["discharge", "charge"]
mass = [2.08, 0.4, 1.36]
colors = mpl.cm.tab10.colors
labels = [r'$\mathrm{1^{st}}$', r'$\mathrm{2^{nd}}$', r'$\mathrm{3^{rd}}$', r'$\mathrm{4^{th}}$']

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, figure=fig)
subfig = fig.add_subfigure(gs[0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

file_seen = {}

for key, df in echem_all.items():
    if not key.endswith(f'_{plot_mode}'):
        continue

    file_idx = int(key.split('_')[0].replace('file', ''))
    cycle_num = int(key.split('_')[1][1:])
    allowed_cycles = selected_cycles_per_file.get(file_idx, [])
    if cycle_num not in allowed_cycles:
        continue

    file_seen[file_idx] = file_seen.get(file_idx, 0)
    label = labels[file_seen[file_idx]] if file_seen[file_idx] < len(labels) else None
    file_seen[file_idx] += 1

    if df.empty:
        continue

    idx_min = df['Ewe/V'].idxmin()
    df_seg = df.loc[:idx_min]
    if df_seg.empty or mass[file_idx] == 0:
        continue

    cap_norm = df_seg['Capacity/mA.h'] * 1000 / mass[file_idx]
    # cap_norm /= cap_norm.max()

    ax.plot(cap_norm, df_seg['Ewe/V'], lw=1.0, c=colors[file_idx], label=label, zorder=1)
    # 添加文件名标签（右下角）
    file_text = "\n".join([f"file{i}: {mass[i]} mg" for i in range(len(mass))])
    ax.text(0.3, 0.02, file_text, transform=ax.transAxes,
            fontsize=9, ha='right', va='bottom', color='k')

# === 6. 格式设置 ===
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11)
ax.set_ylabel(r"$\mathrm{Voltage \ (V \ vs. \ Zn/Zn^{2+})}$", fontsize=11)
# ax.set_xlim(0, 1.5)
# ax.set_ylim(0.9, 1.8)
ax.tick_params(axis='both', direction='out', labelsize=9)
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))

for x in [0.64, 0.88]:
    ax.axvline(x=x, ls='--', lw=0.5, color='k', alpha=0.5)

# ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), frameon=False, labelcolor='linecolor', fontsize=9)



# 可选说明
ax.text(0.05, 0.95,
        r'$\mathrm{pH=4.2,\ ZnSO_4+MnSO_4}$',
        ha='left', va='top', transform=ax.transAxes, fontsize=9, color='k')

# === 7. 保存 ===
name_base = f'Echem_{plot_mode}_with_labels'
for ext in ['tif', 'png']:
    fig.savefig(path_out / f'{name_base}.{ext}', dpi=300, bbox_inches='tight', pad_inches=0.05,
                transparent=False, pil_kwargs={"compression": "tiff_lzw"} if ext == 'tif' else None)

plt.show()


##### FreshElectrolyte

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\FreshElectrolyte\1M ZnSO4+0.2M MnSO4\Results\2025-03').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
path_filelist # 确认对应的信息
echem = [echem[0], echem[1], echem[2], echem[3], echem[5], echem[6], echem[7], echem[4]]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r"#1", None, None, None], [r"#2", None, None, None], [r"#3", None, None, None], [r"#4", None, None, None]] 
for i, data in enumerate(echem[0:4]):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp[temp['Current/uA']<0]['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)

for i, data in enumerate(echem[4:]):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp[temp['Current/uA']<0]['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='--', lw=1.0, c=colors(i), label=None, zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='--', lw=1.0, c=colors(i), label=None, zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 900)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=180, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=90, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=0.05))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.4, 0.6, r'$\mathrm{C/10, 1M \ ZnSO_4 + 0.2M \ MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_FreshElectrolyte_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_FreshElectrolyte_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_FreshElectrolyte_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_FreshElectrolyte_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting -Bulkcell - 1M ZnSO4+0.2M MnSO4 -1

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\BulkCell\1M ZnSO4 + 0.2M MnSO4\Results\2024-ICMAB\3-BulkCell-01C-CP-1M ZnSO4+02M MnSO4-aMnO2-1_6mg').glob(r'*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None, decimal='.').dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']] = df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']].apply(pd.to_numeric, errors='coerce')
    df['time/s'] = df['time/s'].apply(pd.to_datetime, format='mixed', errors='coerce')
    df['cycle number'] = df['cycle number'].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1^{st}}$', None, None, None], [r'$\mathrm{2^{nd}}$', None, None, None], [r'$\mathrm{3^{rd}}$', None, None, None], [r'$\mathrm{4^{th}}$', None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        temp = temp[temp['Ewe/V'] > -0.1]
        # 找到电压最小值的索引
        idx_min = temp['Ewe/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min, 'Capacity/mA.h']*1000/1.6, temp.loc[:idx_min, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=labels[j][i], zorder=0) # labels[i][j], colors[i]
        ax.plot(temp.loc[idx_min+40:, 'Capacity/mA.h']*1000/1.6, temp.loc[idx_min+40:, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=None, zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 320)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Ag/AgCl)", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.25, 0.75)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.05))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.07, r'$\mathrm{Bulk \ Cell, 1M \ ZnSO_4 + 0.2M \ MnSO_4, C/10}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MZn_0.2MMn_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MZn_0.2MMn_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MZn_0.2MMn_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MZn-0.2MMn_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting -Bulkcell - 1M Na2SO4+0.2M MnSO4 -4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\BulkCell\1M Na2SO4 + 0.2M MnSO4\Results\2025-ICMAB\11-BulkCell-03C-CP-1M Na2SO4+02M MnSO4-alphaMnO2-0_4mg-Ar').glob(r'*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None, decimal='.').dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']] = df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']].apply(pd.to_numeric, errors='coerce')
    df['time/s'] = df['time/s'].apply(pd.to_datetime, format='mixed', errors='coerce')
    df['cycle number'] = df['cycle number'].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1^{st}}$', None, None, None], [r'$\mathrm{2^{nd}}$', None, None, None], [r'$\mathrm{3^{rd}}$', None, None, None], [r'$\mathrm{4^{th}}$', None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        temp = temp[temp['Ewe/V'] > -0.1]
        # 找到电压最小值的索引
        idx_min = temp['Ewe/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min, 'Capacity/mA.h']*1000/0.4, temp.loc[:idx_min, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=labels[j][i], zorder=0) # labels[i][j], colors[i]
        ax.plot(temp.loc[idx_min+40:, 'Capacity/mA.h']*1000/0.4, temp.loc[idx_min+40:, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=None, zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
# ax.set_xlim(0, 250)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25, offset=0))

ax.set_ylabel(r"Voltage (V vs. Ag/AgCl)", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.15, 0.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.15, r'$\mathrm{pH: 3.80 \ to \ 3.74}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.05, 0.07, r'$\mathrm{Bulk \ Cell + Ar, 1M \ Na_2SO_4 + 0.2M \ MnSO_4, C/3}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_4_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_4_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_4_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa-0.2MMn_4_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting -Bulkcell - 1M Na2SO4+0.2M MnSO4 -3

In [ ]:
# 电化学上的时间
path_filelist = list(path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\BulkCell\1M Na2SO4 + 0.2M MnSO4\Results\2024-ICMAB\17-BulkCell-02C-CP-1M Na2SO4+02M MnSO4-aMnO2-1_52mg").glob(r'*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None, decimal=',').dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']] = df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']].apply(pd.to_numeric, errors='coerce')
    df['time/s'] = df['time/s'].apply(pd.to_datetime, format='mixed', errors='coerce')
    df['cycle number'] = df['cycle number'].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1^{st}}$', None, None, None], [r'$\mathrm{2^{nd}}$', None, None, None], [r'$\mathrm{3^{rd}}$', None, None, None], [r'$\mathrm{4^{th}}$', None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # temp = temp[temp['Ewe/V'] > -0.02]
        # 找到电压最小值的索引
        idx_min = temp['Ewe/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min, 'Capacity/mA.h']*1000/1.52, temp.loc[:idx_min, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=labels[j][i], zorder=0) # labels[i][j], colors[i]
        ax.plot(temp.loc[idx_min+40:, 'Capacity/mA.h']*1000/1.52, temp.loc[idx_min+40:, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=None, zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 250)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25, offset=0))

ax.set_ylabel(r"Voltage (V vs. Ag/AgCl)", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.6, 0.4)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=0))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.07, r'$\mathrm{Bulk \ Cell, 1M \ Na_2SO_4 + 0.2M \ MnSO_4, C/5}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_3_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_3_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_3_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa-0.2MMn_3_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting -Bulkcell - 1M Na2SO4+0.2M MnSO4 -2

In [ ]:
# 电化学上的时间
path_filelist = list(path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\BulkCell\1M Na2SO4 + 0.2M MnSO4\Results\2024-ICMAB\10-BulkCell-01C-CP-1M Na2SO4+02M MnSO4-aMnO2-1_04mg").glob(r'*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None, decimal=',').dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']] = df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']].apply(pd.to_numeric, errors='coerce')
    df['time/s'] = df['time/s'].apply(pd.to_datetime, format='mixed', errors='coerce')
    df['cycle number'] = df['cycle number'].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1^{st}}$', None, None, None], [r'$\mathrm{2^{nd}}$', None, None, None], [r'$\mathrm{3^{rd}}$', None, None, None], [r'$\mathrm{4^{th}}$', None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        temp = temp[temp['Ewe/V'] > -0.02]
        # 找到电压最小值的索引
        idx_min = temp['Ewe/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min, 'Capacity/mA.h']*1000/1.04, temp.loc[:idx_min, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=labels[j][i], zorder=0) # labels[i][j], colors[i]
        ax.plot(temp.loc[idx_min+40:, 'Capacity/mA.h']*1000/1.04, temp.loc[idx_min+40:, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=None, zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
# ax.set_xlim(0, 500)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=100, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=50, offset=0))

ax.set_ylabel(r"Voltage (V vs. Ag/AgCl)", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.1, 0.5)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05,  offset=0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.07, r'$\mathrm{Bulk \ Cell, 1M \ Na_2SO_4 + 0.2M \ MnSO_4, C/10}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa-0.2MMn_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting -Bulkcell - 1M Na2SO4+0.2M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r"C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\BulkCell\1M Na2SO4 + 0.2M MnSO4\Results\2024-ICMAB\1-BulkCell-01C-CP-1M NaSO4+02M MnSO4- aMnO2-1.04mg").glob(r'*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None, decimal=',').dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']] = df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']].apply(pd.to_numeric, errors='coerce')
    df['time/s'] = df['time/s'].apply(pd.to_datetime, format='mixed', errors='coerce')
    df['cycle number'] = df['cycle number'].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1^{st}}$', None, None, None], [r'$\mathrm{2^{nd}}$', None, None, None], [r'$\mathrm{3^{rd}}$', None, None, None], [r'$\mathrm{4^{th}}$', None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Ewe/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min, 'Capacity/mA.h']*1000/1.04, temp.loc[:idx_min, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=labels[j][i], zorder=0) # labels[i][j], colors[i]
        ax.plot(temp.loc[idx_min+40:, 'Capacity/mA.h']*1000/1.04, temp.loc[idx_min+40:, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=None, zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 500)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=100, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=50, offset=0))

ax.set_ylabel(r"Voltage (V vs. Ag/AgCl)", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.1, 0.9)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.07, r'$\mathrm{Bulk \ Cell, 1M \ Na_2SO_4 + 0.2M \ MnSO_4, C/10}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa_0.2MMn_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_BulkCell_1MNa-0.2MMn_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting - Buffers -Bulkcell -Siavash

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\BulkCell\BufferingElectrolyte\1M ZnSO4 + 0.2M MnSO4\SK-pH Buffer-Zn-1M ZnSO4 + 0.4M MnSO4-HAC-NaAC-CarbonFelt\Results').glob(r'*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None, decimal=',').dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']] = df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']].apply(pd.to_numeric, errors='coerce')
    df['time/s'] = df['time/s'].apply(pd.to_datetime, format='mixed', errors='coerce')
    df['cycle number'] = df['cycle number'].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]

# 选择第三个数据
echem = echem[2:3]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1^{st}}$', None, None, None], [r'$\mathrm{2^{nd}}$', None, None, None], [r'$\mathrm{3^{rd}}$', None, None, None], [r'$\mathrm{4^{th}}$', None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_max = temp['Ewe/V'].idxmax()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_max, 'Capacity/mA.h'], temp.loc[:idx_max, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=labels[j][i], zorder=0, alpha=1-0.8*i) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_max+2:, 'Capacity/mA.h'], temp.loc[idx_max+2:, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=None, zorder=0, alpha=1-0.8*i)

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mA \ h_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 2.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.75, 1.95)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(0.4, 0.6), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{Bulk Cell, ACETATE BUFFER 1 M}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_3_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_3_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_3_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_3_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting - Buffers -Bulkcell

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\BulkCell\BufferingElectrolyte\1M ZnSO4 + 0.2M MnSO4\LC-pH Buffer-1M ZnSO4 + 0.2M MnSO4-HAC-NaAC-4.19-αMnO2-nmg-12cm-8112\Results').glob(r'*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None, decimal=',').dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']] = df[['Ewe/V', 'Ece/V', '<I>/mA','Capacity/mA.h']].apply(pd.to_numeric, errors='coerce')
    df['cycle number'] = df['cycle number'].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1^{st}}$', None, None], [r'$\mathrm{2^{nd}}$', None, None], [r'$\mathrm{3^{rd}}$', None, None], [r'$\mathrm{4^{th}}$', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_max = temp['Ewe/V'].idxmax()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_max, 'Capacity/mA.h']*1000/1.04, temp.loc[:idx_max, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=labels[j][i], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_max+2:, 'Capacity/mA.h']*1000/1.04, temp.loc[idx_max+2:, 'Ewe/V'], ls='-', lw=1.0, c=colors(j), label=None, zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 200)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=40, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=20, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.7, 1.9)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.9), ncols=2, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.15, r'$\mathrm{C/10, 1M \ ZnSO_4 + 0.2M \ MnSO_4}$' + '\n' + r'$\mathrm{Bulk Cell, Buffered, pH=4.2}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting - Buffers

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\pHBuffer\HAC-NaAC-pH4.2\1M ZnAC2\2024-ICMAB').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r"$\mathrm{1^{st}}$", None, None, None], [r"$\mathrm{2^{nd}}$", None, None, None], [r"$\mathrm{3^{rd}}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp[temp['Current/uA']<0]['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(j), label=labels[j][i], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(j), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 250)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=0.05))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.5, 0.4, r'$\mathrm{C/10, 1M \ Zn(CH_3COO)_2,}$' + '\n' + r'$\mathrm{Buffered, pH=4.2}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Buffers_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting - DifferentElectrolytes - 1M MgSO4+02M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\DifferentElectrolytes\1M MgSO4+02M MnSO4\Zn-αMnO2\Results').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r"#1", None, None, None], [r"#2", None, None, None], [r"#3", None, None, None], [r"#4", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp[temp['Current/uA']<0]['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 200)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=40, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=20, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=0.05))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.4, 0.2, r'$\mathrm{C/10, 1M \ MgSO_4 + 0.2M \ MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_DifferentElectrolytes_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_DifferentElectrolytes_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_DifferentElectrolytes_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_DifferentElectrolytes_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting - DifferentElectrolytes - 1M Na2SO4+02M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\DifferentElectrolytes\1M Na2SO4+02M MnSO4\Zn-αMnO2\Results').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r"#1", None, None, None], [r"#2", None, None, None], [r"#3", None, None, None], [r"#4", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp[temp['Current/uA']<0]['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 600)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=100, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=50, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=0.05))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.9, r'$\mathrm{C/10, 1M \ Na_2SO_4 + 0.2M \ MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_DifferentElectrolytes_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_DifferentElectrolytes_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_DifferentElectrolytes_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_DifferentElectrolytes_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### PassivatedZnRichCoatting - Low Cutoff

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\PassivatedZnRichCoatting\LowCutOff\1M ZnSO4+0.2M MnSO4\Results\2025-01').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r"#1", None, None, None], [r"#2", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+3:, 'SpeCap/mAh/g'], temp.loc[idx_min+3:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 500)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=100, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=50, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.0, 1.5)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=0))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.1, r'$\mathrm{C/10, 1M \ ZnSO_4 + 0.2M \ MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Low_Cutoff_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Low_Cutoff_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Low_Cutoff_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_Low_Cutoff_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

#### OrganicEchem

##### StandardElectrolyte - 1M Zn(OTf)2 in DMSO + H2O

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\StandardElectrolyte\ZnOTf2-DMSO-H2O (138)\Results').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [0, 1]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO + H_2O}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+3:, 'SpeCap/mAh/g'], temp.loc[idx_min+3:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 160)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=40, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=20, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.73), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.6, 0.2, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_StandardElectrolyte_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_StandardElectrolyte_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_StandardElectrolyte_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_StandardElectrolyte_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### StandardElectrolyte - 1M ZnSO4 + 0.2M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\StandardElectrolyte\1M ZnSO4 + 0.2M MnSO4\Results').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    if i == 0:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([1,3,4])]
    else:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [1, 0]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 320)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.7), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.1, 0.10, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_StandardElectrolyte_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_StandardElectrolyte_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_StandardElectrolyte_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_StandardElectrolyte_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 2nd0.9V - ZHS - 1M ZnSO4 + 0.2M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\2nd0.9V\ZHS\1M ZnSO4 + 0.2M MnSO4\Results\10#-03-2023-B').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
path_filelist
# # 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
# indices = [0, 1, 2]  # 指定新顺序
# echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 500)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=100, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=50, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.73), ncols=1, frameon=False, labelcolor='linecolor', fontsize=8)
ax.text(0.1, 0.10, r'$\mathrm{ZHS, C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_2nd0.9V_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_2nd0.9V_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_2nd0.9V_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_2nd0.9V_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1st1.80V - ZHS - 1M ZnSO4 + 0.2M MnSO4 -2

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\1st1.80V\ZHS\1M ZnSO4 + 0.2M MnSO4\Results\15#-03-2023-2').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    if i == 2:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,3])]
    else:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
        
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
path_filelist
# # 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
# indices = [0, 1, 2]  # 指定新顺序
# echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 450)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=90, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=45, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.73), ncols=1, frameon=False, labelcolor='linecolor', fontsize=8)
ax.text(0.42, 0.10, r'$\mathrm{ZHS, C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_5_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_5_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_5_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_5_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1st1.80V - ZHS - 1M ZnSO4 + 0.2M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\1st1.80V\ZHS\1M ZnSO4 + 0.2M MnSO4\Results\15#-03-2023').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [0, 1, 2]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO + H_2O}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 450)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=90, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=45, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.65), ncols=1, frameon=False, labelcolor='linecolor', fontsize=8)
ax.text(0.42, 0.08, r'$\mathrm{ZHS, C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_4_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_4_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_4_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_4_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1st1.80V - ZHS - 1M ZnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\1st1.80V\ZHS\1M ZnSO4\Results\15#-03-2023').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [0, 1, 2]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO + H_2O}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 300)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=60, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=30, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.70), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.4, 0.15, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.4, 0.05, r'ZHS', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_3_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_3_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_3_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_3_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1st1.80V - No ZHS - 1M ZnSO4 + 0.2M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\1st1.80V\No ZHS\1M ZnSO4 + 0.2M MnSO4\Results\15#-03-2023').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [0, 1, 2]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO + H_2O}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 350)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=70, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=35, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.65), ncols=1, frameon=False, labelcolor='linecolor', fontsize=8)
ax.text(0.45, 0.15, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.45, 0.05, r'No ZHS', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1st1.80V - No ZHS - 1M ZnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\1st1.80V\No ZHS\1M ZnSO4\Results\15#-03-2023').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [0, 1, 2]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO + H_2O}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 450)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=90, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=45, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.70), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.45, 0.15, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.45, 0.05, r'No ZHS', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st1.80V_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1st0.9V - ZHS - 1M ZnSO4 + 0.2M MnSO4 -2

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\1st0.9V\ZHS\1M ZnSO4 + 0.2M MnSO4\Results\28#-04-2023-B').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
   #  echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
# path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [0, 1, 2]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 400)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.72), ncols=1, frameon=False, labelcolor='linecolor', fontsize=8)
ax.text(0.05, 0.15, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.05, 0.05, r'ZHS', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_5_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_5_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_5_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_5_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1st0.9V - ZHS - 1M ZnSO4 + 0.2M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\1st0.9V\ZHS\1M ZnSO4 + 0.2M MnSO4\Results\15#-03-2023').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
# path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [0, 1, 2]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO + H_2O}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 320)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.9), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.9, 0.15, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.6, 0.05, r'ZHS', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_4_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_4_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_4_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_4_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1st0.9V - ZHS - 1M ZnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\1st0.9V\ZHS\1M ZnSO4\Results\15#-03-2023').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
# path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [0, 1, 2]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO + H_2O}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 450)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=90, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=45, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.9), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.9, 0.15, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.6, 0.05, r'ZHS', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_3_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_3_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_3_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_3_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1st0.9V - No ZHS - 1M ZnSO4 + 0.2M MnSO4 -2

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\1st0.9V\No ZHS\1M ZnSO4 + 0.2M MnSO4\Results\28#-04-2023-B').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
# path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [0, 1, 2]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 350)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=70, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=35, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.7), ncols=1, frameon=False, labelcolor='linecolor', fontsize=8)
ax.text(0.9, 0.15, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.65, 0.05, r'No ZHS', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')


# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1st0.9V - No ZHS - 1M ZnSO4 + 0.2M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\OrganicEchem\1st0.9V\No ZHS\1M ZnSO4 + 0.2M MnSO4\Results\24#-04-2023-B').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
# path_filelist
# 调整电话学数据的顺序，满足，首先制备对应的样品，然后再有机电解液，然后再含水的有机电解液中的反应
indices = [1, 2, 0]  # 指定新顺序
echem = [echem[i] for i in indices]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO}$", None, None, None], [r"$\mathrm{1M \ Zn(OTf)_2 \ in \ DMSO + H_2O}$", None, None, None]] 
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 320)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.9), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.9, 0.15, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.65, 0.05, r'No ZHS', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')


# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_ST_OrganicEchem_1st0.9V_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

#### GCD

##### 1M ZnSO4 + 1M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\1M ZnSO4 + 1M MnSO4').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'#1', None, None, None], [r"#2", None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 320)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.45)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.20, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.05, 0.10, r'$\mathrm{1M \ ZnSO_4 + 1M \ MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_1M_MnSO4_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_1M_MnSO4_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_1M_MnSO4_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_1M_MnSO4_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1M ZnSO4

KDE 密度分布图

In [ ]:
# 读取数据
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\1M ZnSO4').glob(r'KDE*.xlsx'))
KDE = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=0, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   KDE.append(df)

# 只用第一个 KDE
KDE = KDE[0:1]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = list(tol_cset('vibrant'))
for i, data in enumerate(KDE):
    x, y = data["loadings (mg/cm2)"], data["capacity(mA h/g)"]
    sns.scatterplot(data=data, x=x, y=y, s=100, linewidth=1, edgecolor='gainsboro', ax=ax, hue="ratios",palette=[colors[1], colors[3]])
    sns.kdeplot(data=data, x=x, y=y, ax=ax, zorder=5, levels=6, hue="ratios", fill=True, legend=False, palette=[colors[1], colors[3]], alpha=0.3)

ax.set_xlabel(r"Mass Loadings (mg cm$^{-2}$)", fontsize=11,)
ax.set_xlim(0.4, 3.4)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=-0.2))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=-0.2))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: "%.1f" % x))
ax.tick_params(axis='x', labelsize=9)

ax.set_ylabel(r'Capacity (mAh g$\mathrm{{^{-1}_{MnO_2}}}$)', fontsize=11,)
ax.set_ylim(0, 700)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=100, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=50, offset=0))
ax.tick_params(axis='y', labelsize=9)
ax.axhline(y=308.0, xmin=0.0, xmax=1.0, ls='--', lw=1.0, c='silver')

# 修改 legend 的 label
handles, labels= ax.get_legend_handles_labels()
legend_labels = [r'$\mathrm{80\alpha}$-$\mathrm{MnO_2}$-$\mathrm{10SuperP}$-$\mathrm{10PVdf}$', r'$\mathrm{80\alpha}$-$\mathrm{MnO_2}$-$\mathrm{9SuperP}$-$\mathrm{9PVdf}$-$\mathrm{2CNTs}$']
ax.legend(handles, legend_labels, loc='upper right', bbox_to_anchor=(0.85, 1.0), ncols=1, frameon=False, fontsize=10, handlelength=1.0, handletextpad=0.4)

ax.text(0.02, 0.10, r'1M ZnSO${_4}$', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=10, c='black', )

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_KDE_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_KDE_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_KDE_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_KDE_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()


典型曲线

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\1M ZnSO4').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'#1', None, None, None], [r"#2", None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 350)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=70, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=35, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.01, 0.20, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.01, 0.10, r'$\mathrm{1M \ ZnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1M ZnSO4 + 0.2M MnSO4

KDE 密度分布图

In [ ]:
# 读取数据
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\1M ZnSO4 + 0.2M MnSO4').glob(r'KDE*.xlsx'))
KDE = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=0, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   KDE.append(df)

# 只用第一个 KDE
KDE = KDE[0:1]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = list(tol_cset('vibrant'))
for i, data in enumerate(KDE):
    x, y = data["loadings (mg/cm2)"], data["capacity(mA h/g)"]
    sns.scatterplot(data=data, x=x, y=y, s=100, linewidth=1, edgecolor='gainsboro', ax=ax, hue="ratios",palette=[colors[1], colors[3]])
    sns.kdeplot(data=data, x=x, y=y, ax=ax, zorder=5, levels=6, hue="ratios", fill=True, legend=False, palette=[colors[1], colors[3]], alpha=0.3)

ax.set_xlabel(r"Mass Loadings (mg cm$^{-2}$)", fontsize=11,)
ax.set_xlim(0.4, 3.4)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=-0.2))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=-0.2))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: "%.1f" % x))
ax.tick_params(axis='x', labelsize=9)

ax.set_ylabel(r'Capacity (mAh g$\mathrm{{^{-1}_{MnO_2}}}$)', fontsize=11,)
ax.set_ylim(0, 600)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=100, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=50, offset=0))
ax.tick_params(axis='y', labelsize=9)
ax.axhline(y=308.0, xmin=0.0, xmax=1.0, ls='--', lw=1.0, c='silver')

# 修改 legend 的 label
handles, labels= ax.get_legend_handles_labels()
legend_labels = [r'$\mathrm{70\alpha}$-$\mathrm{MnO_2}$-$\mathrm{30CNTs}$', r'$\mathrm{80\alpha}$-$\mathrm{MnO_2}$-$\mathrm{9SuperP}$-$\mathrm{9PVdf}$-$\mathrm{2CNTs}$']
ax.legend(handles, legend_labels, loc='upper right', bbox_to_anchor=(0.85, 1.0), ncols=1, frameon=False, fontsize=10, handlelength=1.0, handletextpad=0.4)

ax.text(0.02, 0.10, r'1M ZnSO${_4}$ + 0.2M MnSO${_4}$', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=10, c='black', )

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_KDE_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_KDE_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_KDE_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_KDE_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()


典型曲线

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\1M ZnSO4 + 0.2M MnSO4').glob(r'LC*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'#1', None, None, None], [r"#2", None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 400)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.20, r'$\mathrm{C/10, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.05, 0.10, r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{1.328 \ mg \ cm^{-2}}$', None, None, None], [r'$\mathrm{1.968 \ mg \ cm^{-2}}$', None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'Capacity/uAh']*2, temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'Capacity/uAh']*2, temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (\mu Ah \ cm^{-2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 800)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=160, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=80, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.75), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.07, r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_2_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_2_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_2_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_2_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

0.02C

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\1M ZnSO4 + 0.2M MnSO4\002C\Results').glob(r'**\*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]


In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'#1', None, None], [r"#2", None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        # ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 400)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.65)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.9), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.20, r'$\mathrm{C/50, 1C = 308 \ mA \ g^{-1}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')
ax.text(0.05, 0.10, r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')


# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_002C_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_002C_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_002C_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_1M_ZnSO4_0.2M_MnSO4_002C_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 0.2M_MnSO4

1st1.80V - 1M ZnSO4 + 0.2M MnSO4 - ZHS

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\0.2M MnSO4\ExtendedCapacity\1st1.80V\1M ZnSO4 + 0.2M MnSO4\Results\ZHS\17#-03-2023').glob(r'**\*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]


In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{0.2M \ MnSO_4, Cycles}$', None, None], [r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$" + "\n" + r"$\mathrm{1^{st} \ Discharge}$", None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 400)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.7), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
# ax.text(0.05, 0.10, r'$\mathrm{1M ZnSO_4 + 0.2M MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_8_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_8_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_8_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_8_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

1st1.80V - 1M ZnSO4 + 0.2M MnSO4 - No ZHS

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\0.2M MnSO4\ExtendedCapacity\1st1.80V\1M ZnSO4 + 0.2M MnSO4\Results\No ZHS\17#-03-2023').glob(r'**\*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]


In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{0.2M \ MnSO_4, Cycles}$', None, None], [r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$" + "\n" + r"$\mathrm{1^{st} \ Discharge}$", None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 400)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.7), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
# ax.text(0.05, 0.10, r'$\mathrm{1M ZnSO_4 + 0.2M MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_7_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_7_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_7_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_7_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

1st1.80V - 1M ZnSO4 - ZHS

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\0.2M MnSO4\ExtendedCapacity\1st1.80V\1M ZnSO4\Results\ZHS\17#-03-2023').glob(r'**\*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2])]
    echem[i] = echem[i][echem[i].iloc[:,2]>=0]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{0.2M \ MnSO_4, Cycles}$', None, None], [r'$\mathrm{1M \ ZnSO_4, 1^{st} \ Discharge}$', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+1:, 'SpeCap/mAh/g'], temp.loc[idx_min+1:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 350)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.7), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_6_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_6_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_6_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_6_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

1st1.80V - 1M ZnSO4 - No ZHS

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\0.2M MnSO4\ExtendedCapacity\1st1.80V\1M ZnSO4\Results\No ZHS\17#-03-2023').glob(r'**\*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{0.2M \ MnSO_4, Cycles}$', None, None], [r'$\mathrm{1M \ ZnSO_4, 1^{st} \ Discharge}$', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min-1, 'SpeCap/mAh/g'], temp.loc[:idx_min-1, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+2:, 'SpeCap/mAh/g'], temp.loc[idx_min+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 350)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.7), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_5_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_5_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_5_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_5_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

1st0.9V - 1M ZnSO4 + 0.2M MnSO4 - ZHS

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\0.2M MnSO4\ExtendedCapacity\1st0.9V\1M ZnSO4 + 0.2M MnSO4\Results\ZHS\17#-03-2023').glob(r'**\*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{0.2M \ MnSO_4, Cycles}$', None, None], [r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$" + "\n" + r"$\mathrm{1^{st} \ Discharge}$", None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min, 'SpeCap/mAh/g'], temp.loc[:idx_min, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+2:, 'SpeCap/mAh/g'], temp.loc[idx_min+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 400)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=8)
# ax.text(0.05, 0.10, r'$\mathrm{1M ZnSO_4 + 0.2M MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_4_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_4_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_4_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_4_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

1st0.9V - 1M ZnSO4 + 0.2M MnSO4 - No ZHS

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\0.2M MnSO4\ExtendedCapacity\1st0.9V\1M ZnSO4 + 0.2M MnSO4\Results\No ZHS\17#-03-2023').glob(r'**\*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{0.2M \ MnSO_4, Cycles}$', None, None], [r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$" + "\n" + r"$\mathrm{1^{st} \ Discharge}$", None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min, 'SpeCap/mAh/g'], temp.loc[:idx_min, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+2:, 'SpeCap/mAh/g'], temp.loc[idx_min+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 400)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.9), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
# ax.text(0.05, 0.10, r'$\mathrm{1M ZnSO_4 + 0.2M MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_3_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_3_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_3_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_3_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

1st0.9V - 1M ZnSO4 - ZHS

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\0.2M MnSO4\ExtendedCapacity\1st0.9V\1M ZnSO4\Results\ZHS\17#-03-2023').glob(r'**\*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{0.2M \ MnSO_4, Cycles}$', None, None], [r'$\mathrm{1M \ ZnSO_4, 1^{st} \ Discharge}$', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min, 'SpeCap/mAh/g'], temp.loc[:idx_min, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+2:, 'SpeCap/mAh/g'], temp.loc[idx_min+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 400)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(0.65, 0.25), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
# ax.text(0.05, 0.10, r'$\mathrm{1M ZnSO_4 + 0.2M MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

1st0.9V - 1M ZnSO4 - No ZHS

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\GCD\0.2M MnSO4\ExtendedCapacity\1st0.9V\1M ZnSO4\Results\No ZHS\17#-03-2023').glob(r'**\*.xlsx'))
echem = []
# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(r"\\?\\" + str(path_filelist[i].resolve()), sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']] = df[[r'Voltage/V',  r'Current/uA',  r'Capacity/uAh',  r'SpeCap/mAh/g']].apply(pd.to_numeric, errors='coerce')
   df['TestTime'] = df['TestTime'].apply(pd.to_timedelta, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1,2,3])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'$\mathrm{0.2M \ MnSO_4, Cycles}$', None, None], [r'$\mathrm{1M \ ZnSO_4, 1^{st} \ Discharge}$', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin()
        # 断开最小值前后的曲线
        ax.plot(temp.loc[:idx_min, 'SpeCap/mAh/g'], temp.loc[:idx_min, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_min+2:, 'SpeCap/mAh/g'], temp.loc[idx_min+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 300)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=-0.15))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=-0.15))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.9), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
# ax.text(0.05, 0.10, r'$\mathrm{1M ZnSO_4 + 0.2M MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_GCD_0.2M_MnSO4_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

#### CV

##### 1M ZnSO4 + 0.2M MnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\CV\1M ZnSO4 + 0.2M MnSO4\2023-ICMAB\Results').glob(r'**\*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None, decimal=',').dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', '<I>/mA']] = df[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
    df['cycle number'] = df['cycle number'].astype(np.int16)  
    echem.append(df)

# 选取每个数据的前三圈
for i in range(len(echem)):
    if i == 0:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([1, 3])]
    # else:
    #     echem[i] = echem[i][echem[i].iloc[:, 0].isin([2])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'0.1 mV/s', None, None], [r'0.3 mV/s', None, None], [r'0.5 mV/s', None, None], [r'0.7 mV/s', None, None], [r'1.0 mV/s', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 断开最小值前后的曲线
        ax.plot(temp['Ewe/V'], temp['<I>/mA'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0.78, 1.82)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=0))

ax.set_ylabel(r"$\mathrm{Current \ (mA \,)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.8, 0.8)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(0.4, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.9, 0.10, r'$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_1M_ZnSO4_0.2M_MnSO4_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_1M_ZnSO4_0.2M_MnSO4_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_1M_ZnSO4_0.2M_MnSO4_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_1M_ZnSO4_0.2M_MnSO4_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 1M ZnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\CV\1M ZnSO4\2023-ICMAB\Results').glob(r'**\*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None, decimal=',').dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', '<I>/mA']] = df[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
    df['cycle number'] = df['cycle number'].astype(np.int16)  
    echem.append(df)

# 选取每个数据的前三圈
for i in range(len(echem)):
    if i == 0:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([1, 2, 3])]
    else:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([2, 3])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'0.1 mV/s', None, None], [r'0.3 mV/s', None, None], [r'0.5 mV/s', None, None], [r'0.7 mV/s', None, None], [r'1.0 mV/s', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 断开最小值前后的曲线
        ax.plot(temp['Ewe/V'], temp['<I>/mA'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0.78, 1.82)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.1,  offset=0))

ax.set_ylabel(r"$\mathrm{Current \ (mA \,)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.8, 0.8)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(0.4, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.9, 0.10, r'$\mathrm{1M \ ZnSO_4}$', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_1M_ZnSO4_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_1M_ZnSO4_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_1M_ZnSO4_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_1M_ZnSO4_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 2M Zn(OTf)2_2

##### 2M ZnSO4_2

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\CV\2M ZnSO4\2022-ICMAB\Results\2022-02-04').glob(r'**\*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', '<I>/mA']] = df[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
    df['cycle number'] = df['cycle number'].astype(np.int16)  
    echem.append(df)

# 选取每个数据的前三圈
for i in range(len(echem)):
    if i == 0:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([1, 2])]
    else:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([2])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'0.1 mV/s', None, None], [r'0.3 mV/s', None, None], [r'0.5 mV/s', None, None], [r'0.7 mV/s', None, None], [r'1.0 mV/s', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 断开最小值前后的曲线
        ax.plot(temp['Ewe/V'], temp['<I>/mA'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0.78, 2.02)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.set_ylabel(r"$\mathrm{Current \ (mA \,)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.6, 0.6)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(0.4, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.9, 0.10, r'$\mathrm{2M \ ZnSO_4}$', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_ZnSO4_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_ZnSO4_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_ZnSO4_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_ZnSO4_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 2M ZnSO4_1

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\CV\2M ZnSO4\2022-ICMAB\Results\2022-01-26').glob(r'**\*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', '<I>/mA']] = df[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
    df['cycle number'] = df['cycle number'].astype(np.int16)  
    echem.append(df)

# 选取每个数据的前三圈
for i in range(len(echem)):
    if i == 0:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([1, 2])]
    else:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([2])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'0.1 mV/s', None, None], [r'0.3 mV/s', None, None], [r'0.5 mV/s', None, None], [r'0.7 mV/s', None, None], [r'1.0 mV/s', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 断开最小值前后的曲线
        ax.plot(temp['Ewe/V'], temp['<I>/mA'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0.78, 2.02)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.set_ylabel(r"$\mathrm{Current \ (mA \,)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(-1.1, 1.7)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.7, offset=0.3))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.35, offset=0.3))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(0.4, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.9, 0.10, r'$\mathrm{2M \ ZnSO_4}$', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_ZnSO4_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_ZnSO4_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_ZnSO4_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_ZnSO4_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 2M Zn(OTf)2_2

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\CV\2M ZnOTf2\2022-ICMAB\Results\2022-02-04').glob(r'**\*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', '<I>/mA']] = df[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
    df['cycle number'] = df['cycle number'].astype(np.int16)  
    echem.append(df)

# 选取每个数据的前三圈
for i in range(len(echem)):
    if i == 0:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([1, 2])]
    else:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([2])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'0.1 mV/s', None, None], [r'0.3 mV/s', None, None], [r'0.5 mV/s', None, None], [r'0.7 mV/s', None, None], [r'1.0 mV/s', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 断开最小值前后的曲线
        ax.plot(temp['Ewe/V'], temp['<I>/mA'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0.78, 2.02)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.set_ylabel(r"$\mathrm{Current \ (mA \,)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.5, 0.5)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(0.4, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.9, 0.10, r'$\mathrm{2M \ Zn(OTf)_2}$', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_Zn(OTf)2_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_Zn(OTf)2_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_Zn(OTf)2_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_Zn(OTf)2_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 2M Zn(OTf)2_1

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\CV\2M ZnOTf2\2022-ICMAB\Results\2022-01-26').glob(r'**\*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', '<I>/mA']] = df[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
    df['cycle number'] = df['cycle number'].astype(np.int16)  
    echem.append(df)

# 选取每个数据的前三圈
for i in range(len(echem)):
    if i == 0:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([1, 2])]
    else:
        echem[i] = echem[i][echem[i].iloc[:, 0].isin([2])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'0.1 mV/s', None, None], [r'0.3 mV/s', None, None], [r'0.5 mV/s', None, None], [r'0.7 mV/s', None, None], [r'1.0 mV/s', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 断开最小值前后的曲线
        ax.plot(temp['Ewe/V'], temp['<I>/mA'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0.78, 2.02)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.set_ylabel(r"$\mathrm{Current \ (mA \,)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.7, 0.8)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(0.4, 1.0), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.9, 0.10, r'$\mathrm{2M \ Zn(OTf)_2}$', ha='right', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_Zn(OTf)2_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_Zn(OTf)2_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_Zn(OTf)2_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_αMnO2_CV_2M_Zn(OTf)2_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

### HT-EMD

#### GCD2

In [ ]:
# 电化学上的时间2
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\EMD\HT-EMD\GCD\1M ZnSO4+02M MnSO4\Results\T11-3mA_6mAh_3cm2').glob(r'**\*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', '<I>/mA']] = df[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
    df['cycle number'] = df['cycle number'].astype(np.int16)  
    echem.append(df)

# 电化学上的时间1
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\EMD\HT-EMD\GCD\1M ZnSO4+02M MnSO4\Results\T11-3mA_6mAh_3cm2').glob(r'**\*.xlsx'))

# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(path_filelist[i], sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df.iloc[:,1:4] = df.iloc[:,1:4].apply(pd.to_numeric, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0, 1])]


In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'3.0 mA Charge', None, None], [r'0.3 mA Discharge', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin() if 'Voltage/V' in temp.columns else temp['Ewe/V'].idxmin()
        # 断开最小值前后的曲线
        if 'Capacity/uAh' in temp.columns:
            ax.plot(temp.loc[:idx_min, 'Capacity/uAh']/1000, temp.loc[:idx_min, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
            # ax.plot(temp.loc[idx_min+2:, 'Capacity/uAh']/1000, temp.loc[idx_min+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)
        elif 'Capacity/mA.h' in temp.columns:
            ax.plot(temp.loc[:idx_min, 'Capacity/mA.h'], temp.loc[:idx_min, 'Ewe/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
            ax.plot(temp.loc[idx_min+2:, 'Capacity/mA.h'], temp.loc[idx_min+2:, 'Ewe/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.1, 6.05)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=1.2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.6, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.6, 2.1)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.5), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{1M ZnSO_4 + 0.2M MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_HT_EMD_GCD_DifferentConcentration_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_HT_EMD_GCD_DifferentConcentration_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_HT_EMD_GCD_DifferentConcentration_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_HT_EMD_GCD_DifferentConcentration_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

#### GCD

In [ ]:
# 电化学上的时间2
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\EMD\HT-EMD\GCD\1M ZnSO4+02M MnSO4\Results\T10-3mA_3mAh_3cm2').glob(r'**\*.txt'))
# 读取电化学数据
echem = []
for path_file in path_filelist:
    line_skip = None
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip = int(line.split(':')[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(path_file, sep='\t', comment='#', skiprows=line_skip-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 
     # # 转换数据格式
    df[['Ewe/V', '<I>/mA']] = df[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
    df['cycle number'] = df['cycle number'].astype(np.int16)  
    echem.append(df)

# 电化学上的时间1
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\EMD\HT-EMD\GCD\1M ZnSO4+02M MnSO4\Results\T10-3mA_3mAh_3cm2').glob(r'**\*.xlsx'))

# 读取电化学数据
for i in range(len(path_filelist)):
   df = pd.read_excel(path_filelist[i], sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df.iloc[:,1:4] = df.iloc[:,1:4].apply(pd.to_numeric, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0, 1])]


In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'3.0 mA Charge', None, None], [r'0.3 mA Discharge', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin() if 'Voltage/V' in temp.columns else temp['Ewe/V'].idxmin()
        # 断开最小值前后的曲线
        if 'Capacity/uAh' in temp.columns:
            ax.plot(temp.loc[:idx_min, 'Capacity/uAh']/1000, temp.loc[:idx_min, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
            # ax.plot(temp.loc[idx_min+2:, 'Capacity/uAh']/1000, temp.loc[idx_min+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)
        elif 'Capacity/mA.h' in temp.columns:
            ax.plot(temp.loc[:idx_min, 'Capacity/mA.h'], temp.loc[:idx_min, 'Ewe/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
            ax.plot(temp.loc[idx_min+2:, 'Capacity/mA.h'], temp.loc[idx_min+2:, 'Ewe/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.01, 3.05)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.5, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.2), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{1M ZnSO_4 + 0.2M MnSO_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_HT_EMD_GCD_DifferentConcentration_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_HT_EMD_GCD_DifferentConcentration_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_HT_EMD_GCD_DifferentConcentration_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_HT_EMD_GCD_DifferentConcentration_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()


### EMD

#### GCD

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\EMD\EMD\GCD\1M ZnSO4 + 0.2M MnSO4\Results\2V-2mAh').glob(r'**\*.xlsx'))

# 读取电化学数据
echem = []
for i in range(len(path_filelist)):
   df = pd.read_excel(path_filelist[i], sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df.iloc[:,1:4] = df.iloc[:,1:4].apply(pd.to_numeric, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第三圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i]['Cycle'].isin([1, 2, 3])]

print(len(echem) == len(path_filelist))

# 排除最开始沉积的数据
echem = echem[0:1]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'1M Zn + 0.2M Mn', None, None], [r'1M Zn + 0.2M Mn', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data['Cycle'].unique()):
        temp = data[data['Cycle'] == idx]
        # 找到电压最小值的索引
        idx_min = temp['Voltage/V'].idxmin() 
        # 断开最小值前后的曲线
        if 'Capacity/uAh' in temp.columns:
            ax.plot(temp.loc[:idx_min, 'Capacity/uAh']/1000, temp.loc[:idx_min, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
            ax.plot(temp.loc[idx_min+2:, 'Capacity/uAh']/1000, temp.loc[idx_min+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)
        else:
            ax.plot(temp.loc[:idx_min, 'Capacity/mAh'], temp.loc[:idx_min, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
            ax.plot(temp.loc[idx_min+2:, 'Capacity/mAh'], temp.loc[idx_min+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.01, 0.15)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.03, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.015, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.6), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{In Total: \ 2 \ mAh}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_EMD_GCD_DifferentConcentration_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_EMD_GCD_DifferentConcentration_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_EMD_GCD_DifferentConcentration_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_EMD_GCD_DifferentConcentration_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()


### Stainless Steel

#### The effects of 1M ZnSO4 +1M MnSO4 on Deposition of MnO2 on Stainless Steel, 0.025mA

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\Substrates\Stainless_Steel\GCD\1M ZnSO4 +1M MnSO4\Results').glob(r'**\*025*.xlsx'))

# 读取电化学数据
echem = []
for i in range(len(path_filelist)):
   df = pd.read_excel(path_filelist[i], sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[['Voltage/V', 'Current/uA', 'Capacity/uAh']] = df[['Voltage/V', 'Current/uA', 'Capacity/uAh']].apply(pd.to_numeric, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第三圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i]['Cycle'].isin([1, 2, 3])]

print(len(echem) == len(path_filelist))

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'1M Zn + 1M Mn', None, None], [r'1M Zn + 1M Mn', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data['Cycle'].unique()):
        temp = data[data['Cycle'] == idx]
        # 找到电压最大值的索引
        idx_max = temp['Voltage/V'].idxmax() 
        # 断开最大值前后的曲线
        ax.plot(temp.loc[:idx_max, 'Capacity/uAh'], temp.loc[:idx_max, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_max+2:, 'Capacity/uAh'], temp.loc[idx_max+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (\mu Ah \ cm^{-2}\,)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.2, 15)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{50 \ \mu A\ cm^{-2}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_SS_GCD_DifferentConcentration_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_SS_GCD_DifferentConcentration_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_SS_GCD_DifferentConcentration_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_SS_GCD_DifferentConcentration_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

### CNTs

#### The effects of 1M ZnSO4 +1M MnSO4 on Deposition of MnO2 on CNTs, 0.025mA

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\Substrates\CNTs\GCD\1M ZnSO4 +1M MnSO4\Results').glob(r'**\*025*.xlsx'))

# 读取电化学数据
echem = []
for i in range(len(path_filelist)):
   df = pd.read_excel(path_filelist[i], sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[['Voltage/V', 'Current/uA', 'Capacity/uAh']] = df[['Voltage/V', 'Current/uA', 'Capacity/uAh']].apply(pd.to_numeric, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第三圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i]['Cycle'].isin([1, 2, 3])]

print(len(echem) == len(path_filelist))

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'1M Zn + 1M Mn', None, None], [r'1M Zn + 1M Mn', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data['Cycle'].unique()):
        temp = data[data['Cycle'] == idx]
        # 找到电压最大值的索引
        idx_max = temp['Voltage/V'].idxmax() 
        # 断开最大值前后的曲线
        ax.plot(temp.loc[:idx_max, 'Capacity/uAh'], temp.loc[:idx_max, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_max+2:, 'Capacity/uAh'], temp.loc[idx_max+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (\mu Ah \ cm^{-2}\,)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.2, 15)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{50 \ \mu A\ cm^{-2}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_CNTs_GCD_DifferentConcentration_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_CNTs_GCD_DifferentConcentration_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_CNTs_GCD_DifferentConcentration_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_CNTs_GCD_DifferentConcentration_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

### rGOs

#### The effects of 1M ZnSO4 +1M MnSO4 on Deposition of MnO2 on GO, 0.025mA

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\Substrates\GO\GCD\1M ZnSO4 +1M MnSO4\Results').glob(r'**\*025*.xlsx'))

# 读取电化学数据
echem = []
for i in range(len(path_filelist)):
   df = pd.read_excel(path_filelist[i], sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[['Voltage/V', 'Current/uA', 'Capacity/uAh']] = df[['Voltage/V', 'Current/uA', 'Capacity/uAh']].apply(pd.to_numeric, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第三圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i]['Cycle'].isin([1, 2, 3])]

print(len(echem) == len(path_filelist))

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'1M Zn + 1M Mn', None, None], [r'1M Zn + 1M Mn', None, None], [r'1M Zn + 1M Mn', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data['Cycle'].unique()):
        temp = data[data['Cycle'] == idx]
        # 找到电压最大值的索引
        idx_max = temp['Voltage/V'].idxmax() 
        # 断开最大值前后的曲线
        ax.plot(temp.loc[:idx_max, 'Capacity/uAh'], temp.loc[:idx_max, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_max+2:, 'Capacity/uAh'], temp.loc[idx_max+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (\mu Ah \ cm^{-2}\,)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.2, 20)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=4, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=2, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.7), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{50 \ \mu A\ cm^{-2}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_GO_GCD_DifferentConcentration_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_GO_GCD_DifferentConcentration_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_GO_GCD_DifferentConcentration_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_GO_GCD_DifferentConcentration_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

### H23

#### CV

##### 2M ZnSO4

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\CarbonPaper_H23\CV\2M ZnSO4\Results\2022-02-05').glob(r'**\*.txt'))

line_skip = []
for path_file in path_filelist:
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip_index = int(line.split(':')[1].strip())
                line_skip.append(line_skip_index)
                break  # 发现后立即退出循环，提高效率


# 读取电化学数据
echemA = pd.read_csv(path_filelist[0], sep='\t', comment='#', skiprows=line_skip[0]-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 
echemB = pd.read_csv(path_filelist[1], sep='\t', comment='#', skiprows=line_skip[1]-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 

# # 转换数据格式
echemA[['Ewe/V', '<I>/mA']] = echemA[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
echemA['cycle number'] = echemA['cycle number'].astype(np.int16)

echemB[['Ewe/V', '<I>/mA']] = echemB[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
echemB['cycle number'] = echemB['cycle number'].astype(np.int16)

# echemA.info(), echemB.info()

In [ ]:
# 选取 第一个数据的第两和三圈，以及第二个数据的第两和三圈
echemA = echemA[echemA['cycle number'].isin([2, 3])]
echemB = echemB[echemB['cycle number'].isin([2, 3])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(echemA['Ewe/V'], echemA['<I>/mA']*1000, ls='-', lw=1.0, c=colors[0], label=r"0.1 mV/s", zorder=0)
ax.plot(echemB['Ewe/V'], echemB['<I>/mA']*1000, ls='-', lw=1.0, c=colors[1], label=r"0.3 mV/s", zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0.78, 2.02)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.set_ylabel(r"$\mathrm{Current \ (\mu A)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(-10, 20)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=6, offset=2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=3, offset=2))

ax.tick_params(axis='both', direction='out', labelsize=9)

ax.legend(loc='upper right', bbox_to_anchor=(1.0, 1), ncols=1, frameon=False, labelcolor='linecolor', fontsize=10)

ax.text(0.05, 0.08, r'2M ZnSO$\mathrm{_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_CV_ZnSO4_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_CV_ZnSO4_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_CV_ZnSO4_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_CV_ZnSO4_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### 2M Zn(OTf)2

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\CarbonPaper_H23\CV\2M ZnOTf2\Results\2022-02-04').glob(r'**\*.txt'))

line_skip = []
for path_file in path_filelist:
    with open(path_file, 'r', encoding='latin_1') as file:
        for line in file:
            if line.startswith('Nb header lines'):
                line_skip_index = int(line.split(':')[1].strip())
                line_skip.append(line_skip_index)
                break  # 发现后立即退出循环，提高效率

print(len(path_filelist))


# 读取电化学数据
echemA = pd.read_csv(path_filelist[0], sep='\t', comment='#', skiprows=line_skip[0]-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 
echemB = pd.read_csv(path_filelist[1], sep='\t', comment='#', skiprows=line_skip[1]-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 
echemC = pd.read_csv(path_filelist[2], sep='\t', comment='#', skiprows=line_skip[2]-1, encoding='latin_1', index_col=None).dropna(axis=1, how='all') 

# # 转换数据格式
echemA[['Ewe/V', '<I>/mA']] = echemA[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
echemA['cycle number'] = echemA['cycle number'].astype(np.int16)

echemB[['Ewe/V', '<I>/mA']] = echemB[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
echemB['cycle number'] = echemB['cycle number'].astype(np.int16)

echemC[['Ewe/V', '<I>/mA']] = echemC[['Ewe/V', '<I>/mA']].apply(pd.to_numeric, errors='coerce')
echemC['cycle number'] = echemC['cycle number'].astype(np.int16)

# echemA.info(), echemB.info(), echemC.info()

In [ ]:
# 选取 第一个数据的第两和三圈，以及第二个数据的第两和三圈
echemA = echemA[echemA['cycle number'].isin([2, 3])]
echemB = echemB[echemB['cycle number'].isin([2, 3])]
echemC = echemC[echemC['cycle number'].isin([2, 3])]

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

ax.plot(echemA['Ewe/V'], echemA['<I>/mA']*1000, ls='-', lw=1.0, c=colors[0], label=r"0.1 mV/s", zorder=0)
ax.plot(echemB['Ewe/V'], echemB['<I>/mA']*1000, ls='-', lw=1.0, c=colors[1], label=r"0.3 mV/s", zorder=0)
ax.plot(echemC['Ewe/V'], echemC['<I>/mA']*1000, ls='-', lw=1.0, c=colors[2], label=r"0.5 mV/s", zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0.78, 2.02)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.set_ylabel(r"$\mathrm{Current \ (\mu A)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(-10, 20)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=6, offset=2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=3, offset=2))

ax.tick_params(axis='both', direction='out', labelsize=9)

ax.legend(loc='upper right', bbox_to_anchor=(0.4, 1), ncols=1, frameon=False, labelcolor='linecolor', fontsize=10)

ax.text(0.7, 0.08, r'2M Zn(OTf$\mathrm{_2})$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_CV_Zn(OTf)2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_CV_Zn(OTf)2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_CV_Zn(OTf)2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_CV_Zn(OTf)2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

#### GCD

##### The effects of DifferentConcentrations on Deposition of MnO2 on Carbon Paper H23, 0.009mA

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\Substrates\CarbonPaper_H23\GCD\DifferentConcentration\Results').glob(r'**\*009*.xlsx'))

# 选取需要的数据
path_filelist = [path_filelist[i] for i in [0,2,3]]

# 读取电化学数据
echem = []
for i in range(len(path_filelist)):
   df = pd.read_excel(path_filelist[i], sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[['Voltage/V', 'Current/uA', 'Capacity/uAh']] = df[['Voltage/V', 'Current/uA', 'Capacity/uAh']].apply(pd.to_numeric, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第三圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i]['Cycle'].isin([1, 2, 3])]

print(len(echem) == len(path_filelist))

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'0M Zn + 1M Mn', None, None], [r'1M Zn + 0M Mn', None, None], [r'1M Zn + 1M Mn', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data['Cycle'].unique()):
        temp = data[data['Cycle'] == idx]
        # 找到电压最大值的索引
        idx_max = temp['Voltage/V'].idxmax() 
        # 断开最大值前后的曲线
        ax.plot(temp.loc[:idx_max, 'Capacity/uAh'], temp.loc[:idx_max, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_max+2:, 'Capacity/uAh'], temp.loc[idx_max+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (\mu Ah \ cm^{-2}\,)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.2, 10)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{18 \ \mu A\ cm^{-2}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_GCD_DifferentConcentration_1_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_GCD_DifferentConcentration_1_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_GCD_DifferentConcentration_1_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_GCD_DifferentConcentration_1_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### The effects of DifferentConcentrations on Deposition of MnO2 on Carbon Paper H23, 0.025mA

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\Substrates\CarbonPaper_H23\GCD\DifferentConcentration\Results').glob(r'**\*025*.xlsx'))

# # 选取需要的数据
# path_filelist = [path_filelist[i] for i in [0,1,3,4,6]] 

# 读取电化学数据
echem = []
for i in range(len(path_filelist)):
   df = pd.read_excel(path_filelist[i], sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[['Voltage/V', 'Current/uA', 'Capacity/uAh']] = df[['Voltage/V', 'Current/uA', 'Capacity/uAh']].apply(pd.to_numeric, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第三圈
for i in range(len(echem)):
   echem[i] = echem[i][echem[i]['Cycle'].isin([1, 2, 3])]

print(len(echem) == len(path_filelist))

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'1M Zn + 1M Mn', None, None], [r'1M Zn + 1M Mn', None, None], [r'1M Zn + 1M Mn', None, None], [r'1M Zn + 1M Mn', None, None], [r'1M Zn + 1M Mn', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data['Cycle'].unique()):
        temp = data[data['Cycle'] == idx]
        # 找到电压最大值的索引
        idx_max = temp['Voltage/V'].idxmax() 
        # 断开最大值前后的曲线
        ax.plot(temp.loc[:idx_max, 'Capacity/uAh'], temp.loc[:idx_max, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_max+8:, 'Capacity/uAh'], temp.loc[idx_max+8:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (\mu Ah \ cm^{-2}\,)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.2, 40)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=8, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=4, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{50 \ \mu A\ cm^{-2}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_GCD_DifferentConcentration_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_GCD_DifferentConcentration_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_GCD_DifferentConcentration_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_H23_GCD_DifferentConcentration_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

### CarbonFelt_Treated-V

#### GCD

##### The effects of DifferentConcentrations on Deposition of MnO2 on CarbonFelt_Treated-V, 0.009mA

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\Substrates\CarbonFelt_Treated-V\GCD\DifferentConcentration\Results').glob(r'**\*009*.xlsx'))

# 选取需要的数据
path_filelist = [path_filelist[i] for i in [1,3,5,6,7]]

# 读取电化学数据
echem = []
for i in range(len(path_filelist)):
   df = pd.read_excel(path_filelist[i], sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[['Voltage/V', 'Current/uA', 'Capacity/uAh']] = df[['Voltage/V', 'Current/uA', 'Capacity/uAh']].apply(pd.to_numeric, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第三圈
for i in range(len(echem)):
   if i == 3:
      echem[i] = echem[i][echem[i]['Cycle'].isin([1, 2])]
   else:
      echem[i] = echem[i][echem[i]['Cycle'].isin([1, 2, 3])]

print(len(echem) == len(path_filelist))

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'0M Zn + 1M Mn', None, None], [r'1M Zn + 0.2M Mn', None, None], [r'1M Zn + 0M Mn', None, None], [r'1M Zn + 1M Mn', None, None], [r'1M Zn + 3M Mn', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data['Cycle'].unique()):
        temp = data[data['Cycle'] == idx]
        # 找到电压最大值的索引
        idx_max = temp['Voltage/V'].idxmax() 
        # 断开最大值前后的曲线
        ax.plot(temp.loc[:idx_max, 'Capacity/uAh'], temp.loc[:idx_max, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_max+2:, 'Capacity/uAh'], temp.loc[idx_max+2:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (\mu Ah \ cm^{-2}\,)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.2, 80)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{18 \ \mu A\ cm^{-2}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_CarbonFelt_Treated-V_GCD_DifferentConcentration_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_CarbonFelt_Treated-V_GCD_DifferentConcentration_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_CarbonFelt_Treated-V_GCD_DifferentConcentration_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_CarbonFelt_Treated-V_GCD_DifferentConcentration_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

##### The effects of DifferentConcentrations on Deposition of MnO2 on CarbonFelt_Treated-V, 0.025mA

In [ ]:
# 电化学上的时间
path_filelist = list(path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\Substrates\CarbonFelt_Treated-V\GCD\DifferentConcentration\Results').glob(r'**\*025*.xlsx'))

# 选取需要的数据
path_filelist = [path_filelist[i] for i in [0,1,3,4,6]] 

# 读取电化学数据
echem = []
for i in range(len(path_filelist)):
   df = pd.read_excel(path_filelist[i], sheet_name=2, engine='openpyxl', comment='#', header=0, index_col=None).dropna(axis=1, how='all')
   # 转换数据格式
   df[['Voltage/V', 'Current/uA', 'Capacity/uAh']] = df[['Voltage/V', 'Current/uA', 'Capacity/uAh']].apply(pd.to_numeric, errors='coerce')
   df['Cycle'] = df['Cycle'].astype(np.int16)
   echem.append(df)

# 选取每个数据的第一到第三圈
for i in range(len(echem)):
   echem[i] = echem[i][echem[i]['Cycle'].isin([1, 2, 3])]

print(len(echem) == len(path_filelist))

In [ ]:
%matplotlib inline
plt.close('all')

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
colors = mpl.cm.tab10 # rainbow(np.linspace(0, 1.0, len(echem)))
labels = [[r'0M Zn + 1M Mn', None, None], [r'1M Zn + 0.2M Mn', None, None], [r'1M Zn + 0M Mn', None, None], [r'1M Zn + 1M Mn', None, None], [r'1M Zn + 3M Mn', None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data['Cycle'].unique()):
        temp = data[data['Cycle'] == idx]
        # 找到电压最大值的索引
        idx_max = temp['Voltage/V'].idxmax() 
        # 断开最大值前后的曲线
        ax.plot(temp.loc[:idx_max, 'Capacity/uAh'], temp.loc[:idx_max, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=labels[i][j], zorder=0) # labels[i][j] colors[i]
        ax.plot(temp.loc[idx_max+8:, 'Capacity/uAh'], temp.loc[idx_max+8:, 'Voltage/V'], ls='-', lw=1.0, c=colors(i), label=None, zorder=0)


# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (\mu Ah \ cm^{-2}\,)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(-0.2, 150)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=30, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=15, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(0.8, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15,  offset=-0.1))

ax.tick_params(axis='both', direction='out', labelsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)
ax.text(0.05, 0.10, r'$\mathrm{50 \ \mu A\ cm^{-2}}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

# 保存图片
plt.savefig(path.joinpath(path_out, r'Ecehm_CarbonFelt_Treated-V_GCD_DifferentConcentration_2_300.tif'), pad_inches=0.05, bbox_inches='tight', dpi=300, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_CarbonFelt_Treated-V_GCD_DifferentConcentration_2_600.tif'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False, pil_kwargs={"compression": "tiff_lzw"})
plt.savefig(path.joinpath(path_out, r'Ecehm_CarbonFelt_Treated-V_GCD_DifferentConcentration_2_600.png'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.savefig(path.joinpath(path_out, r'Ecehm_CarbonFelt_Treated-V_GCD_DifferentConcentration_2_600.pdf'), pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
# plt.gcf().set_facecolor('white') 
plt.show()

### 项目一 - Mn-Zn

#### 常规 Echem，GCD 数据

In [ ]:
# 数据导入
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Uno\Result\Echem\αMnO2\SpecialTest\DifferentElectrolyte\1M MgSO4+02M MnSO4\2024-ICMAB')
file = pd.read_csv(path.joinpath(path_file, r'Export20240909160754.csv'), index_col=None, header=0)
file.head(1)

In [ ]:
# 画图
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图 A
subfig_a = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig_a.add_axes((0, 0, 1.0, 1.0), zorder=0)
ax.plot(file.iloc[:, 2], file.iloc[:, 0], ls='-', lw=1.0, c=colors[0], label=r"$\mathrm{1^{\!st}}$ Discharge")

ax.set_xlabel(r'Capacity (mAh g$\mathrm{^{-1}_{MnO_2}}$)', fontsize=11,)
ax.set_xlim(0, 500)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=100))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=50))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}$)", fontsize=11,)
ax.set_ylim(0.85, 1.3)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05))

ax.tick_params(axis='x', labelsize=9)
ax.tick_params(axis='y', labelsize=9)

ax.legend(loc='upper right', bbox_to_anchor=(1.0, 1), ncols=1, frameon=False, labelcolor='linecolor', fontsize=9)

ax.text(0.01, 0.08, r'1M MgSO$\mathrm{_4}$ + 0.2M MnSO$\mathrm{_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')
# ax.text(0.01, 0.08, r'1M Na$\mathrm{_2}$SO$\mathrm{_4}$ + 0.2M MnSO$\mathrm{_4}$', ha='left', va='top', transform=ax.transAxes, fontsize=9, c='k')

plt.savefig(path.joinpath(path_out, r'1_Mn_Zn_Echem_GCD_1.tif'), 
            pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.show()

### 项目二 - Cellulose

#### BC@SuperP@aMnO2, GCD

In [ ]:
# 读取数据
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Cuatro\Result\Echem\αMnO2@SuperP+Bacterial Cellulose\40%_Bacterial Cellulose\GCD\1M ZnSO4 + 0.2M MnSO4\2024-ICMAB\1SuperP-2αMnO2')
file = pd.read_excel(path.joinpath(path_file, r'alphaMnO2-1-2-2.2mg_20240812150151_DefaultGroup_01_7.xlsx'), sheet_name=2, skiprows=0, header=0, comment='#')
file.head(1)

In [ ]:
# 选择不同的 cycles
%matplotlib inline
plt.close('all')

fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None,
                        wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position([0, 0, 1.0, 1.0])

cycle_i = file[file['Cycle'] == 1]
index_i = cycle_i['Voltage/V'].idxmin()
ax.plot(cycle_i['SpeCap/mAh/g'].iloc[:index_i-1], cycle_i['Voltage/V'].iloc[:index_i-1], color=colors[0], label=r'$1^\mathrm{st}$')
ax.plot(cycle_i['SpeCap/mAh/g'].iloc[index_i+1:], cycle_i['Voltage/V'].iloc[index_i+1:], color=colors[0], label=None)

cycle_num = np.where(file['Cycle'].value_counts().shape[0]-1>30, 30, file['Cycle'].value_counts().shape[0]-1)
labels = [None]*int(cycle_num)
labels[0], labels[1] = r'$2^\mathrm{nd}$', r'$3^\mathrm{rd}$'

if len(labels) > 5:
    labels[4] = r'$5^\mathrm{th}$'    
if len(labels) > 10:
    labels[9] = r'$10^\mathrm{th}$'

for i in range(cycle_num):
    
    cycle_i = file[file['Cycle'] == i+2]
    index_i = np.where(cycle_i['SpeCap/mAh/g'].argsort().iloc[0]>cycle_i['SpeCap/mAh/g'].argsort().iloc[1], cycle_i['SpeCap/mAh/g'].argsort().iloc[0], cycle_i['SpeCap/mAh/g'].argsort().iloc[1])
    ax.plot(cycle_i['SpeCap/mAh/g'].iloc[:index_i-1], cycle_i['Voltage/V'].iloc[:index_i-1], color=colors[(i+2)%len(colors)], label=labels[i])
    ax.plot(cycle_i['SpeCap/mAh/g'].iloc[index_i+1:], cycle_i['Voltage/V'].iloc[index_i+1:], color=colors[(i+2)%len(colors)], label=None)
    
ax.set_xlabel(r'Capacity (mAh g$\mathrm{^{-1}_{MnO2}}$)', fontsize=11)
ax.set_xlim(0, 250)
ax.set_ylabel(r'Voltage (V)', fontsize=11)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: "%.2f" % y))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25))
ax.legend(loc='upper left', bbox_to_anchor=(0.75, 0.5), ncols=1, frameon=False,
          labelcolor='linecolor', fontsize=9)

plt.savefig(path.joinpath(path_out, r'1_BC_Echem_GCD_5.tif'), 
            pad_inches=0.05, bbox_inches='tight', dpi=600, transparent=False)
plt.show()

#### BC@SuperP，EIS

In [ ]:
# 读取数据
path_file = path(r'C:\Users\chengliu\OneDrive - UAB\ICMAB-Data\Zn-Mn\Cuatro\Result\Echem\SuperP+Bacterial Cellulose\Zn\Conductivity+TransferNumber\2M ZnSO4\2024-ICMAB\A-Zn-Zn')
file = pd.read_csv(path.joinpath(path_file, r'Results', r'EIS_Zn_Zn_Conductivity.csv'), sep=',', skiprows=1, header=0, comment='#')
file.head(1)

In [ ]:
%matplotlib inline

plt.close('all')
fig = plt.figure(figsize=(9, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1,1,1], height_ratios=None,
                        wspace=0, hspace=0, figure=fig)

# 图 A
subfiga = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfiga.add_axes((0, 0, 1, 1),zorder=0)

labels_a = [r'Glass Fiber', r'SP_0%', r'SP_10%', r'SP_20%', r'SP_40%', r'SP_60%', r'SP_80%', r'SP_100%']

for i in range(len(labels_a)):
    ax.scatter(file.iloc[:, 2*i], file.iloc[:, 2*i+1], c=colors[i])
    ax.plot(file.iloc[:, 2*i], file.iloc[:, 2*i+1], c=colors[i], ls='-', lw=1.0, label=labels_a[i])
      
ax.set_xlim(0, 2000)
ax.set_xlabel(r'Re(Z) (ohm)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=500))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=250))
ax.set_ylim(0, 2000)
ax.set_ylabel(r"-Im(Z) (ohm)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=500))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=250))
ax.tick_params(axis='x', labelsize=9) 
ax.tick_params(axis='y', labelsize=9)
ax.legend(loc='upper left', bbox_to_anchor=(0.65, 1), ncols=1, frameon=False,
              labelcolor='linecolor', fontsize=8)
axins = ax.inset_axes([0.08, 0.4, 0.55, 0.55], zorder=3,)
for i in range(len(labels_a)-4):
    j=i+4
    axins.scatter(file.iloc[:, 2*j], file.iloc[:, 2*j+1], c=colors[j], s=15)
    axins.plot(file.iloc[:, 2*j], file.iloc[:, 2*j+1], c=colors[j], ls='-', lw=1.0, label=labels_a[j])
    axins.set_xlim(0, 12.5)
    axins.xaxis.set_major_locator(ticker.MultipleLocator(base=2))
    axins.xaxis.set_minor_locator(ticker.MultipleLocator(base=1))
    axins.set_ylim(0, 3)
    axins.yaxis.set_major_locator(ticker.MultipleLocator(base=1))
    axins.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
    axins.tick_params(axis='x', labelsize=9) 
    axins.tick_params(axis='y', labelsize=9)
    axins.legend(loc='upper left', bbox_to_anchor=(0.01, 1), ncols=1, frameon=False,
              labelcolor='linecolor', fontsize=7)

# 图 B
subfigb = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfigb.add_axes((0.25, 0, 1, 1),zorder=0)

data_b = pd.concat([file.iloc[:, 6:8], file.iloc[:, 10:12], file.iloc[:, 16:20]], axis=1, ignore_index=True)
labels_b = [r'SP_20%', r'SP_60%', r'SP_20%_air', r'SP_60%_air']

for i in range(len(labels_b)):
    ax.scatter(data_b.iloc[:, 2*i], data_b.iloc[:, 2*i+1], c=colors[i])
    ax.plot(data_b.iloc[:, 2*i], data_b.iloc[:, 2*i+1], c=colors[i], ls='-', lw=1.0, label=labels_b[i])
    
ax.set_xlim(0, 350)
ax.set_xlabel(r'Re(Z) (ohm)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=50))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=25))
ax.set_ylim(0, 350)
ax.set_ylabel(r"-Im(Z) (ohm)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=50))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=25))
ax.tick_params(axis='x', labelsize=9) 
ax.tick_params(axis='y', labelsize=9)

ax.legend(loc='upper left', bbox_to_anchor=(0.63, 1), ncols=1, frameon=False,
              labelcolor='linecolor', fontsize=8)

axins = ax.inset_axes([0.10, 0.4, 0.55, 0.55], zorder=3,)
for i in range(len(labels_b)-2):
    j = 4*i+2
    axins.scatter(data_b.iloc[:, j], data_b.iloc[:, j+1], c=colors[2*i+1], s=15)
    axins.plot(data_b.iloc[:, j], data_b.iloc[:, j+1], c=colors[2*i+1], ls='-', lw=1.0, label=labels_b[2*i+1])
    axins.set_xlim(0, 12.5)
    axins.xaxis.set_major_locator(ticker.MultipleLocator(base=2))
    axins.xaxis.set_minor_locator(ticker.MultipleLocator(base=1))
    axins.set_ylim(0, 3)
    axins.yaxis.set_major_locator(ticker.MultipleLocator(base=1))
    axins.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
    axins.tick_params(axis='x', labelsize=9) 
    axins.tick_params(axis='y', labelsize=9)
    axins.legend(loc='upper left', bbox_to_anchor=(0.01, 1), ncols=1, frameon=False,
              labelcolor='linecolor', fontsize=7)

# 图 C
subfiga = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfiga.add_axes((0.45, 0, 1, 1),zorder=0)

ax.scatter(file.iloc[:, 8], file.iloc[:, 9], c=colors[4], alpha=0.5)
ax.plot(file.iloc[:, 8], file.iloc[:, 9], c=colors[4], ls='-', lw=1.0, label=labels_a[4], alpha=0.5)
ax.scatter(file.iloc[:, -2], file.iloc[:, -1], c=colors[4])
ax.plot(file.iloc[:, -2], file.iloc[:, -1], c=colors[4], ls='-', lw=1.0, label=r'SP_40%_2')

      
ax.set_xlim(2, 7.5)
ax.set_xlabel(r'Re(Z) (ohm)', fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=1))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
ax.set_ylim(0, 1.0)
ax.set_ylabel(r"-Im(Z) (ohm)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(axis='x', labelsize=9) 
ax.tick_params(axis='y', labelsize=9)
ax.legend(loc='upper left', bbox_to_anchor=(0.02, 1), ncols=1, frameon=False,
              labelcolor='linecolor', fontsize=8)

plt.savefig(path.joinpath(path_out, r'1_Cellulose_Echem_EIS.tif'), transparent=False,
            pad_inches=0.05, bbox_inches='tight', dpi=300)

plt.show()

### 项目三 - Poland 数据

In [ ]:
# 读取电化学数据
file_folder = path(r'C:\Users\chengliu\Desktop\Poland_II\RawData\09-10-2024\Echem')
filelist = list(file_folder.glob(r'*.txt'))

In [ ]:
df_all = pd.DataFrame()
for file in filelist:
    with open(file, 'r', encoding='utf-8') as f:
        number2index = int(re.findall(r'\d+', str(f.readlines(50)))[0])
        line_index=0
        for line in f:
            line_index += 1
            if line_index == number2index-2:
                # print(line.rstrip('\n'), type(line))
                break
        name = line.split()

        df = pd.read_csv(file, sep='\s+', index_col=False, header=None, skiprows=number2index,  encoding='utf-8',) # 
        df = df.loc[:, (df!=0).any(axis=0)]
        df.insert(2, r'A', (df.iloc[:, 0] + ' ' + df.iloc[:, 1]).values)
        df = df.iloc[:,2:].copy()
        df.rename(columns=dict(zip(df.columns, name)), errors='raise', inplace=True)
        df_all = pd.concat([df_all, df], axis=0, ignore_index=True,)
        
# df_all['time/s'] = pd.to_datetime(df_all['time/s'])
# df_all.sort_values(by='time/s')
# C2mAh = df_all['(Q-Qo)/C']/3.6
# C2mAh.rename(r'(Q-Qo)/mA.h', inplace=True)
# df_all.loc[C2mAh.index[C2mAh.notna()], '(Q-Qo)/mA.h'] = C2mAh[C2mAh.notna()]
# df_all.pop('(Q-Qo)/C')
# df_all.to_csv(path.joinpath(file.parent, r'Echem_all.csv'), index=None, header=True)
# # display(df_all.head(2))
# print(r'Done') 

In [ ]:
# df_all = pd.DataFrame()
# for file in filelist:
#     with open(file, 'r', encoding='utf-8') as f:
#         number2index = int(re.findall(r'\d+', str(f.readlines(50)))[0])
#         line_index=0
#         for line in f:
#             line_index += 1
#             if line_index == number2index-2:
#                 # print(line.rstrip('\n'), type(line))
#                 break
#         name = line.split()

#         df = pd.read_csv(file, sep='\s+', index_col=False, header=None, skiprows=number2index,  encoding='utf-8',) # 
#         df = df.loc[:, (df!=0).any(axis=0)]
#         df.insert(2, r'A', (df.iloc[:, 0] + ' ' + df.iloc[:, 1]).values)
#         df = df.iloc[:,2:].copy()
#         df.rename(columns=dict(zip(df.columns, name)), errors='raise', inplace=True)
#         df_all = pd.concat([df_all, df], axis=0, ignore_index=True,)
        
# df_all['time/s'] = pd.to_datetime(df_all['time/s'])
# df_all.sort_values(by='time/s')
# C2mAh = df_all['(Q-Qo)/C']/3.6
# C2mAh.rename(r'(Q-Qo)/mA.h', inplace=True)
# df_all.loc[C2mAh.index[C2mAh.notna()], '(Q-Qo)/mA.h'] = C2mAh[C2mAh.notna()]
# df_all.pop('(Q-Qo)/C')
# df_all.to_csv(path.joinpath(file.parent, r'Echem_all.csv'), index=None, header=True)
# # display(df_all.head(2))
# print(r'Done') 